Part 1: Word Embedding Arithmetic (30 Marks)
Task
Create 5 examples of word arithmetic similar to the "king - man + woman ≈ queen" analogy. Use
words that have relevant semantic relationships.
Steps
1. Load the BERT model and tokenizer.
2. Implement functions to get word embeddings and perform word arithmetic.
3. Write word_arithmetic and find_most_similar functions to create your examples
4. The word arithmetic function will be able to take two list of words:
○ The first list is parameters to the word_arithmatic as example, (paris, france,
italy), run the arithmetic and collect the return value (e.g., paris - france + italy =
?).
○ Using the find_most_similar function with return value of word_arithmetic
as input, along with the second list of words like (rome, romaine, ramania, ronnie,
random) to find the most similar word to the answer.
○ Show this for of 5 potential pairs of such words
○ Print answer for each of the 5 test cases.



In [1]:
!pip install transformers torch gensim numpy scipy matplotlib

In [2]:
import torch
from transformers import AutoTokenizer, AutoModel
import gensim.downloader as api
import numpy as np
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt

In [5]:
# Load pre-trained BERT model and tokenizer using Auto classes
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')


In [6]:
from transformers import AutoTokenizer, AutoModel
import torch
from scipy.spatial.distance import cosine

# Load pre-trained BERT model and tokenizer using Auto classes
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Function to get the word embedding from BERT
def get_word_embedding(word):
    # Tokenize the input word and get the embeddings
    tokens = tokenizer(word, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**tokens)
    # Take the mean of the token embeddings
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
    return embedding

# Function to perform word arithmetic
def word_arithmetic(word1, word2, word3):
    vec1 = get_word_embedding(word1)
    vec2 = get_word_embedding(word2)
    vec3 = get_word_embedding(word3)

    # Perform arithmetic: word1 - word2 + word3
    result_vec = vec1 - vec2 + vec3
    return result_vec

# Function to find the most similar word
def find_most_similar(target_vec, word_list):
    similarities = []
    for word in word_list:
        word_vec = get_word_embedding(word)
        # Calculate cosine similarity
        similarity = 1 - cosine(target_vec, word_vec)
        similarities.append((word, similarity))
    # Sort by similarity
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[0]  # Return the most similar word and its similarity score

# New examples of word arithmetic
examples = [
    ('king', 'man', 'queen', ['woman', 'prince', 'child', 'royalty', 'emperor']),
    ('woman', 'mother', 'father', ['parent', 'sister', 'brother', 'teacher', 'daughter']),
    ('sun', 'day', 'moon', ['night', 'star', 'light', 'eclipse', 'solar']),
    ('car', 'driver', 'pilot', ['plane', 'vehicle', 'boat', 'engineer', 'mechanic']),
    ('teacher', 'student', 'school', ['classroom', 'education', 'principal', 'course', 'homework'])
]

# Perform word arithmetic and find the most similar word for each example
for word1, word2, word3, options in examples:
    result_emb = word_arithmetic(word1, word2, word3)
    most_similar, similarity = find_most_similar(result_emb, options)
    print(f"{word1} - {word2} + {word3} is most similar to: {most_similar} (similarity: {similarity:.4f})")


king - man + queen is most similar to: prince (similarity: 0.8199)
woman - mother + father is most similar to: brother (similarity: 0.8887)
sun - day + moon is most similar to: star (similarity: 0.7528)
car - driver + pilot is most similar to: vehicle (similarity: 0.8589)
teacher - student + school is most similar to: education (similarity: 0.7746)


Part 2: RAG System Implementation (30 Marks)
Task
Implement a simple RAG system using LangChain, process an article of your choice, and run 5
different queries on its content.
Steps
1. Choose at least 5 diverse articles on a different topic of your interest from wikipedia
dump on HuggingFace (e.g., Artificial Intelligence, Machine Learning, etc.).
2. Use the provided code from the class to load and process each article, create
embeddings, store embeddings for each article in the single VectorDB and set up the
RAG system.
3. Formulate 10 diverse queries that explore various aspects of your article's content.
4. Run each query using the run_query function and record the results.


In [7]:
!pip install langchain groq
!pip install langchain-groq groq
!pip install -U langchain-community
!pip install langchain langchain-community huggingface_hub faiss-cpu
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/

In [12]:
!pip install wikipedia-api



In [13]:
!pip install langchain langchain_groq faiss-cpu transformers


In [19]:
!pip install wikipedia


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=f1a3eae54a45a8ad3af9a137c1cde78e252517ce690475c878956fa0945ff28c
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [20]:
import wikipedia


In [21]:
# Install necessary packages
!pip install wikipedia-api langchain langchain_groq faiss-cpu transformers

# Import necessary libraries
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI  # Assuming ChatGroq works similarly
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WikipediaLoader
import os

# Set Groq API key
os.environ["GROQ_API_KEY"] = "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"  # Replace with your Groq API key

# Step 1: Choose 5 articles (article titles are just examples, adjust as needed)
articles = [
    "Artificial Intelligence",
    "Quantum Computing",
    "Climate Change",
    "Ancient Civilizations",
    "Space Exploration"
]

# Step 2: Load and process each article
documents = []
for article_title in articles:
    loader = WikipediaLoader(article_title)
    article_text = loader.load()
    documents.extend(article_text)

# Step 3: Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs_chunks = text_splitter.split_documents(documents)

# Step 4: Create embeddings and store in a VectorDB
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
vector_db = FAISS.from_documents(docs_chunks, embedding_model)

# Step 5: Initialize the Groq LLM
llm = ChatGroq(model_name="mixtral-8x7b-32768")  # Ensure ChatGroq is compatible with LangChain

# Step 6: Load the QA chain using the appropriate LLM
qa_chain = load_qa_chain(llm, chain_type="stuff")

# Step 7: Define the query function using the chain and retriever
def run_query(query):
    docs = vector_db.similarity_search(query)
    result = qa_chain.run(input_documents=docs, question=query)
    return result

# Step 8: Run 10 diverse queries on the RAG system
queries = [
    "What are the latest advancements in quantum computing?",
    "How does climate change affect ocean currents?",
    "What are the key features of ancient Egyptian architecture?",
    "How have space missions contributed to our understanding of the solar system?",
    "What are the primary differences between supervised and unsupervised machine learning?",
    "Can you explain the concept of genetic modification and its applications?",
    "What are the environmental impacts of deforestation on wildlife?",
    "How did the Industrial Revolution change societal structures?",
    "What are the challenges and benefits of renewable energy adoption?",
    "What is the significance of dark matter in modern physics?"
]

# Step 9: Run each query and record results
for i, query in enumerate(queries, 1):
    response = run_query(query)
    print(f"Query {i}: {query}")
    print(f"Response: {response}\n")


/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
<ipython-input-21-2b242ecc4198>:38: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingf

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-21-2b242ecc4198>:45: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  qa_chain = load_qa_chain(llm, chain_type="stuff")
<ipython-input-21-2b242ecc4198>:50: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa_chain.run(input_documents=docs, question=query)


Query 1: What are the latest advancements in quantum computing?
Response: The latest advancements in quantum computing include the merger of Cambridge Quantum and Honeywell Quantum Solutions to form Quantinuum, a quantum computing company. Quantinuum's H-Series trapped-ion quantum computers set the highest quantum volume to date of 1,048,576 in April 2024. The company has also developed middleware and software products for various applications such as cybersecurity, quantum chemistry, quantum machine learning, and quantum artificial intelligence. Additionally, Quantinuum offers quantum-computing-hardened encryption keys for data protection.

In terms of quantum computing technology, Quantinuum's architecture supports all-to-all qubit connectivity, enabling entangled states to be created between all qubits and providing a high fidelity of quantum states.

These advancements show progress in the development of quantum computing, with potential applications in various fields such as crypt